## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-10-05-48-29 +0000,startribune,NBA veteran Julius Randle finds an inner peace...,https://www.startribune.com/julius-randle-timb...
1,2026-01-10-05-46-58 +0000,nyt,Washington National Opera Is Leaving the Kenne...,https://www.nytimes.com/2026/01/09/arts/music/...
2,2026-01-10-05-45-24 +0000,nyt,"Thomas V. Cash, Cartel-Busting D.E.A. Chief in...",https://www.nytimes.com/2026/01/09/us/thomas-v...
3,2026-01-10-05-39-35 +0000,nyt,Venezuela Updates: Trump’s Push to Invest in V...,https://www.nytimes.com/live/2026/01/09/world/...
4,2026-01-10-05-09-34 +0000,nyt,Landfill Collapse in Philippines Leaves 4 Dead...,https://www.nytimes.com/2026/01/09/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2368/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
24,trump,68
22,venezuela,23
135,ice,23
30,oil,20
193,iran,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
72,2026-01-10-00-23-02 +0000,nypost,President Trump wears ‘Happy Trump’ lapel pin ...,https://nypost.com/2026/01/09/us-news/presiden...,169
3,2026-01-10-05-39-35 +0000,nyt,Venezuela Updates: Trump’s Push to Invest in V...,https://www.nytimes.com/live/2026/01/09/world/...,149
274,2026-01-09-12-23-44 +0000,nyt,Trump Indicates He Will Meet Venezuela’s Macha...,https://www.nytimes.com/2026/01/09/world/ameri...,127
46,2026-01-10-01-40-34 +0000,nyt,Trump’s $100 Billion Venezuela Oil Plan Gets a...,https://www.nytimes.com/2026/01/09/business/en...,126
323,2026-01-09-09-00-00 +0000,cbc,How Trump and Vance's accounts of Minneapolis ...,https://www.cbc.ca/news/world/minneapolis-ice-...,126


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
72,169,2026-01-10-00-23-02 +0000,nypost,President Trump wears ‘Happy Trump’ lapel pin ...,https://nypost.com/2026/01/09/us-news/presiden...
48,111,2026-01-10-01-35-55 +0000,nypost,Anti-ICE lefties insist Renee Nicole Good was ...,https://nypost.com/2026/01/09/us-news/lefties-...
3,61,2026-01-10-05-39-35 +0000,nyt,Venezuela Updates: Trump’s Push to Invest in V...,https://www.nytimes.com/live/2026/01/09/world/...
78,59,2026-01-10-00-05-41 +0000,bbc,South Africa's strained ties with US face new ...,https://www.bbc.com/news/articles/c62wxezynk2o...
38,54,2026-01-10-02-00-00 +0000,wsj,America’s unemployment rate is low. But so is ...,https://www.wsj.com/economy/jobs/job-market-co...
190,47,2026-01-09-18-46-01 +0000,nypost,Sergey Brin joins fellow Google co-founder Lar...,https://nypost.com/2026/01/09/business/fellow-...
28,41,2026-01-10-02-53-01 +0000,nypost,"Trump calls for a 10%, one-year cap on credit ...",https://nypost.com/2026/01/09/us-news/trump-ca...
80,38,2026-01-10-00-01-00 +0000,wsj,Wall Street’s move after the 2008 housing cras...,https://www.wsj.com/economy/housing/trump-blin...
74,37,2026-01-10-00-19-00 +0000,wsj,Pair Shot by Border Patrol in Portland Linked ...,https://www.wsj.com/us-news/two-people-shot-by...
252,37,2026-01-09-14-42-40 +0000,nypost,Italian PM Giorgia Meloni doubts Trump will us...,https://nypost.com/2026/01/09/world-news/melon...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
